In [6]:
#Importando as libs usadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
#Lendo o dataset
input_df = pd.read_csv('C:/Users/estev/Desktop/desafio_part_binahki/Parte 2/Input.csv', sep=';')
valid_df = pd.read_csv('C:/Users/estev/Desktop/desafio_part_binahki/Parte 2/Validação v2.csv', encoding='unicode_escape', sep=';')

In [9]:
#Verificando os dados input e como estão dispostos
input_df

,DATA,corrente_neutro,corrente_f1,corrente_f2,corrente_f3,tensao_f1,tensao_f1_f2,tensao_f2
0,23/03/2022 11:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23/03/2022 10:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23/03/2022 10:59,9.53,4.16,3.35,2.02,254.2,440.29,254.0
3,23/03/2022 10:59,9.56,4.17,3.35,2.03,254.1,440.11,253.9
4,23/03/2022 10:59,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
601,23/03/2022 07:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
602,23/03/2022 07:31,9.44,4.12,3.33,1.98,252.1,436.65,251.8
603,23/03/2022 07:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
604,23/03/2022 07:30,9.36,4.07,3.34,1.95,251.8,436.13,251.6


In [10]:
#Removendo os valores NaN's pelo fato de que 
input_df.dropna(inplace = True)

In [11]:
#Alterando o título DATA para datetime e alterando o título tensao_f1_f2 para tensao_linha_f1_f2 que é a tensão
#entre fases
input_df.rename({'DATA':'datetime', 'tensao_f1_f2':'tensao_linha_f1_f2'}, axis=1, inplace=True)

In [12]:
#Organizando os dados por data e hora
input_org = input_df.sort_values(by="datetime")

In [13]:
#Removendo a coluna de tensão de linha, pois não seria tão interessante pra gente 
input_org.drop(['tensao_linha_f1_f2'], axis = 1, inplace = True)

In [15]:
#Levando em consideração que as tensões tem as mesmas amplitudes e frequências, temos
input_org['tensao_f3'] = input_org['tensao_f1']

In [18]:
#Adicionando as infos de ângulo de fases
input_org['angulo_tensao_f1'] = 0
input_org['angulo_tensao_f2'] = 119.7
input_org['angulo_tensao_f3'] = -120.1
input_org['fp'] = 0.86

In [19]:
#Visualizando os dados já organizados
input_org

,datetime,corrente_neutro,corrente_f1,corrente_f2,corrente_f3,tensao_f1,tensao_f2,tensao_f3,angulo_tensao_f1,angulo_tensao_f2,angulo_tensao_f3,fp
604,23/03/2022 07:30,9.36,4.07,3.34,1.95,251.8,251.6,251.8,0,119.7,-120.1,0.86
602,23/03/2022 07:31,9.44,4.12,3.33,1.98,252.1,251.8,252.1,0,119.7,-120.1,0.86
599,23/03/2022 07:32,9.49,4.13,3.36,2.00,253.4,253.3,253.4,0,119.7,-120.1,0.86
596,23/03/2022 07:33,9.48,4.10,3.32,2.06,253.4,253.2,253.4,0,119.7,-120.1,0.86
593,23/03/2022 07:34,9.32,4.04,3.24,2.04,253.0,252.8,253.0,0,119.7,-120.1,0.86
...,...,...,...,...,...,...,...,...,...,...,...,...
11,23/03/2022 10:56,9.38,4.08,3.27,2.02,253.1,252.9,253.1,0,119.7,-120.1,0.86
8,23/03/2022 10:57,9.48,4.13,3.28,2.06,253.8,253.5,253.8,0,119.7,-120.1,0.86
6,23/03/2022 10:58,9.45,4.12,3.31,2.02,254.1,253.9,254.1,0,119.7,-120.1,0.86
3,23/03/2022 10:59,9.56,4.17,3.35,2.03,254.1,253.9,254.1,0,119.7,-120.1,0.86


In [20]:
#Novo dataframe
report_df = input_org.filter(items=['datetime'])

In [21]:
#Antes de começarmos os cálculos das potências precisamos saber a informação de ângulo de fase da corrente
#Considerando que as cargas são indutivas temos o fp atrasado
#Como FP = Cos(theta) e theta = (theta v) - (theta i), ficamos com:
theta = np.degrees(np.arccos(input_org['fp']))
#Logo, os angulos de fase das correntes
theta_i_f1 = input_org['angulo_tensao_f1'] - theta
theta_i_f2 = input_org['angulo_tensao_f2'] - theta
theta_i_f3 = input_org['angulo_tensao_f3'] - theta
#print(theta)
#print(theta_i_f1)
#print(theta_i_f2)
#print(theta_i_f3)

In [22]:
#Calculando o terceiro termo da equação P = V(rms)*I(rms)*cos(theta(v)-theta(i))
cos_theta_f1 = np.cos(np.radians(input_org['angulo_tensao_f1'] - theta_i_f1))
cos_theta_f2 = np.cos(np.radians(input_org['angulo_tensao_f2'] - theta_i_f2))
cos_theta_f3 = np.cos(np.radians(input_org['angulo_tensao_f3'] - theta_i_f3))

#print(cos_theta_f1)
#print(cos_theta_f2)
#print(cos_theta_f3)

In [23]:
# Considerando a Potência ativa para o regime permanente senoidal, temos P = V(rms)*I(rms)*cos(theta(v)-theta(i))
potencia_ativa_f1 = input_org['corrente_f1']*input_org['tensao_f1']*cos_theta_f1
potencia_ativa_f2 = input_org['corrente_f2']*input_org['tensao_f2']*cos_theta_f2
potencia_ativa_f3 = input_org['corrente_f2']*input_org['tensao_f2']*cos_theta_f3

#print(potencia_ativa_f1)
#print(potencia_ativa_f2)
#print(potencia_ativa_f3)

In [24]:
#Calculando os consumos para 1 min, que é o tempo de cada datetime
energia_ativa_f1 = potencia_ativa_f1/1000*0.017
energia_ativa_f2 = potencia_ativa_f2/1000*0.017
energia_ativa_f3 = potencia_ativa_f3/1000*0.017

#energia_ativa_total
energia_ativa_total = energia_ativa_f1 + energia_ativa_f2 + energia_ativa_f3

#print(energia_ativa_f1)
#print(energia_ativa_f2)
#print(energia_ativa_f3)
#print(energia_ativa_total)

In [25]:
#Criando títulos no dataframe
report_df['fator_potencia'] = input_org['fp']
report_df['energia_ativa_f1_(KWh)'] = energia_ativa_f1
report_df['energia_ativa_f2_(KWh)'] = energia_ativa_f2
report_df['energia_ativa_f3_(KWh)'] = energia_ativa_f3
report_df['energia_ativa_total_(KWh)'] = energia_ativa_total

In [26]:
#Calculando o terceiro termo da equação Q = V(rms)*I(rms)*sin(theta(v)-theta(i))
sin_theta_f1 = np.sin(np.radians(input_org['angulo_tensao_f1']) - theta_i_f1)
sin_theta_f2 = np.sin(np.radians(input_org['angulo_tensao_f2']) - theta_i_f2)
sin_theta_f3 = np.sin(np.radians(input_org['angulo_tensao_f3']) - theta_i_f3)

In [27]:
#Calculando a potência reativa
potencia_reativa_f1 = input_org['corrente_f1']*input_org['tensao_f1']*sin_theta_f1
potencia_reativa_f2 = input_org['corrente_f2']*input_org['tensao_f2']*sin_theta_f2
potencia_reativa_f3 = input_org['corrente_f2']*input_org['tensao_f2']*sin_theta_f3

In [28]:
#Calculando os consumos para 1 min, que é o tempo de cada datetime
energia_reativa_f1 = potencia_reativa_f1/1000*0.017
energia_reativa_f2 = potencia_reativa_f2/1000*0.017
energia_reativa_f3 = potencia_reativa_f3/1000*0.017
energia_reativa_total = energia_reativa_f1 + energia_reativa_f2 + energia_reativa_f3

In [29]:
#Criando títulos no dataframe
report_df['energia_reativa_f1_(KVArh)'] = energia_reativa_f1
report_df['energia_reativa_f2(KVArh)'] = energia_reativa_f2
report_df['energia_reativa_f3(KVArh)'] = energia_reativa_f3
report_df['energia_reativa_total_(KVArh)'] = energia_reativa_total

In [30]:
#Calculos de energia aparente
energia_aparente_f1 = ((potencia_ativa_f1/input_org['fp'])/1000)*0.017
energia_aparente_f2 = ((potencia_ativa_f2/input_org['fp'])/1000)*0.017
energia_aparente_f3 = ((potencia_ativa_f3/input_org['fp'])/1000)*0.017
energia_aparente_total = energia_ativa_f1 + energia_aparente_f2 + energia_aparente_f3

In [31]:
#Criando títulos no dataframe
report_df['energia_aparente_f1_(KVAh)'] = energia_ativa_f1
report_df['energia_aparente_f2(KVAh)'] = energia_aparente_f2
report_df['energia_aparente_f3(KVAh)'] = energia_aparente_f3
report_df['energia_aparente_total_(KVAh)'] = energia_aparente_total

In [32]:
report_df

,datetime,fator_potencia,energia_ativa_f1_(KWh),energia_ativa_f2_(KWh),energia_ativa_f3_(KWh),energia_ativa_total_(KWh),energia_reativa_f1_(KVArh),energia_reativa_f2(KVArh),energia_reativa_f3(KVArh),energia_reativa_total_(KVArh),energia_aparente_f1_(KVAh),energia_aparente_f2(KVAh),energia_aparente_f3(KVAh),energia_aparente_total_(KVAh)
604,23/03/2022 07:30,0.86,0.014983,0.012286,0.012286,0.039555,-0.011651,0.012300,-0.012265,-0.011616,0.014983,0.014286,0.014286,0.043555
602,23/03/2022 07:31,0.86,0.015185,0.012259,0.012259,0.039703,-0.011808,0.012273,-0.012238,-0.011773,0.015185,0.014254,0.014254,0.043694
599,23/03/2022 07:32,0.86,0.015300,0.012443,0.012443,0.040186,-0.011898,0.012457,-0.012422,-0.011863,0.015300,0.014468,0.014468,0.044237
596,23/03/2022 07:33,0.86,0.015189,0.012290,0.012290,0.039769,-0.011811,0.012304,-0.012269,-0.011777,0.015189,0.014291,0.014291,0.043771
593,23/03/2022 07:34,0.86,0.014943,0.011975,0.011975,0.038893,-0.011620,0.011988,-0.011955,-0.011586,0.014943,0.013924,0.013924,0.042792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,23/03/2022 10:56,0.86,0.015097,0.012090,0.012090,0.039278,-0.011740,0.012104,-0.012070,-0.011706,0.015097,0.014059,0.014059,0.043215
8,23/03/2022 10:57,0.86,0.015325,0.012156,0.012156,0.039637,-0.011916,0.012170,-0.012136,-0.011882,0.015325,0.014135,0.014135,0.043595
6,23/03/2022 10:58,0.86,0.015306,0.012287,0.012287,0.039879,-0.011902,0.012301,-0.012266,-0.011867,0.015306,0.014287,0.014287,0.043879
3,23/03/2022 10:59,0.86,0.015491,0.012435,0.012435,0.040362,-0.012046,0.012449,-0.012414,-0.012011,0.015491,0.014460,0.014460,0.044411


In [ ]:
#cuspindo o arquivo em csv
report_df.to_csv('circuito_estrela.csv', index=False)